In [38]:
reset()
%display latex
Partitions.options.latex="list"
from sage.misc.cachefunc import cached_method, cached_function
from sage.structure.unique_representation import UniqueRepresentation
from itertools import chain, combinations
from collections import deque

load("partition_4d.sage")



#KT_vars = ['x1','x2','x3','x4','y','u1','u2','u3','u4','w']
KT_vars = ['x1','x2','x3','x4','y']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)


coho_vars = ['t1','t2','t3','t4','m']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)


#x1,x2,x3,x4,y,u1,u2,u3,u4,w=KT_ring.gens()
x1,x2,x3,x4,y=KT_ring.gens()
t1,t2,t3,t4,m= coho_ring.gens()
b=var('b')

pP4ds = Partitions4d()

def character(p,w=(x1,x2,x3,x4)):
    Nx1, Nx2, Nx3, Nx4 = p._Nx1, p._Nx2, p._Nx3, p._Nx4
    return specialize(substitute(sum(x1^(i) * x2^(j) * x3^(k) * x4^(l)
           for (i,j,k,l) in p.boxes(Nx1=Nx1, Nx2=Nx2, Nx3=Nx3, Nx4=Nx4)),w))

def dual(f):
    if f in ZZ:
        return f
    R = KT_ring
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g


def specialize(f):
    if f in ZZ:
        return f
    return f(x1=x1^2,x2=x2^2,x3=x3^2,x4=x4^2)

def degn(f,n,b=b):
    g=f.taylor(b,0,n)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

def deg0(f):
    if f==0: 
        return 0
    return pretty(degn(f(t1=b*t1,t2=b*t2,t3=b*t3,t4=b*t4,m=b*m),0))


def fixed_locus(n):
    return list(pP4ds.with_num_boxes(n))
    
    
def sqrtK(w):
    K=x1*x2*x3*x4
    return sqrt(specialize(substitute(K,w)))

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def Tvir(p,w=(x1,x2,x3,x4)):
    Q = character(p,w)
    Q_inv =dual(Q)
    K=sqrtK(w)
    P=(1-x1)*(1-x2)*(1-x3)*(1-x4)
    P=specialize(P)
    P=substitute(P,w)
    T = K*(1-(1-P*Q)*(1-dual(P*Q)))/P
    return T
    #return T.numerator()*dual(T.denominator())

def sqrtTvir(p,w=(x1,x2,x3,x4)):
    Q =character(p,w)
    Q_inv = dual(Q)
    P=(1-x1)*(1-x2)*(1-x3)*(1-x4)
    P=specialize(P)
    P=substitute(P,w)
    P0=substitute(specialize(dual((1-x1)*(1-x2)*(1-x3))),w)
    K=sqrtK(w)
    sqrtT = K*(Q-P0*Q*Q_inv)
    return sqrtT

def sqrtmono(f):
    numer, denom = 1,1
    for exp, coeff in f.dict().items():
        items=deque(exp)
        items.rotate(len(coho_vars))
        term = KT_ring({tuple(list(items)):1})
        numer *= term ** (coeff)
    return numer

def NekBracket(f):
    if f.parent()==ZZ:
        return 0
    R = f.parent()
    numer, denom = 1,1
    for exp, coeff in f.dict().items():
        if exp.is_constant():
            term = R.zero()
        else:
            items=deque(exp)
            items.rotate(len(coho_vars))
            term = KT_ring({tuple(list(items)):1})*(1-KT_ring({exp.emul(-1):1}))
        if coeff > 0:
            numer *= term ** (coeff)
        elif coeff < 0:
            denom *= term ** (-coeff)
    return R.fraction_field()(numer) / denom

def Nek(n,w=(x1,x2,x3,x4)):
    total=0
    for SP in fixed_locus(n):
        add=SP.sign()*\
        NekBracket(dual(specialize(character(SP,w))*y^(-1))-sqrtTvir(SP,w))
        add=add(x1=1,x2=1,x4=1,u1=1,u2=1,u3=1,u4=1)(x3=1)
        total+=add
    return total

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def cnkn(n,w=(x1,x2,x3,x4)):
    total=0
    for SP in fixed_locus(n):
        add=SP.sign()*top_chern(character(SP,w)*sqrtK(w))
        add=add/top_chern(sqrtTvir(SP,w))
        total+=add
    return total



def F(v=(x1,x2,x3,x4)):
    return specialize(substitute((x1^-1+x2^-1+x3^-1+x4^-1-x1-x2-x3-x4)/(1-x1)/(1-x2)/(1-x3)/(1-x4)*NekBracket(y)/(1-b*w)/(1-b*w^-1),v))


def F_list(w):
    return sum(F(w[i]) for i in range(8))


def sqrt_to_original(f):
    R = f.parent()
    term=0
    for exp, coeff in f.dict().items():
        newexp=()
        for i in range(len(coho_vars)):
            newexp+=(exp[i]+exp[i+len(coho_vars)]//2,)
        for i in range(len(coho_vars)):
            newexp+=(exp[len(coho_vars)+i]%2,)
        term +=R({newexp:coeff})
    return term
    
def cut(f):
    return sqrt_to_original(f.numerator())/sqrt_to_original(f.denominator())
    
def substitute(f,w):
    return f(x1=w[0],x2=w[1],x3=w[2],x4=w[3])

def substitute_coho(f,w):
    return f(t1=from_monomial(w[0]),t2=from_monomial(w[1]),t3=from_monomial(w[2]),t4=from_monomial(w[3]))

def Nek_list(n,w,sgn):
    total=0
    for i in range(8):
        add=sgn[i]*(Nek(n,w[i]))
        total+=add
    return total

def cn_list(n,w,sgn):
    total=0
    for i in range(8):
        add=sgn[i]*substitute_coho(cnkn(n),w[i])
        total+=add
    return total


def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
    
v=[(x1,x2,x3,x4),
   (x1,1/x2,x3/x2,x4/x2),
   (x1,1/x3,x2/x3,x4/x3),
   (x1,1/x4,x2/x4,x3/x4),
   (1/x1,x2,x3,x4),
   (1/x1,1/x2,x3/x2,x4/x2),
   (1/x1,1/x3,x2/x3,x4/x3),
   (1/x1,1/x4,x2/x4,x3/x4),
  ]

sgn=()

def genbin(n, bs=()):
    global sgn
    if len(bs) == n:
        sgn=sgn+(bs, )
    else:
        genbin(n, bs + (1,))
        genbin(n, bs + (-1,))

genbin(8,(1,))

In [19]:
from_monomial(x1/x2)

t1 - t2

In [53]:
sqrtTvir(fixed_locus(1)[0],v[4])

x1*x2*x3*x4 - x1*x2*x3^-1*x4 - x1*x2^-1*x3*x4 + x1*x2^-1*x3^-1*x4 + x1^-1*x2*x3^-1*x4 + x1^-1*x2^-1*x3*x4 - x1^-1*x2^-1*x3^-1*x4

In [47]:
str(denominator(cn_list(1,v,sgn[1])))

't1^14 - 15*t1^12*t2^2 + 69*t1^10*t2^4 - 155*t1^8*t2^6 + 195*t1^6*t2^8 - 141*t1^4*t2^10 + 55*t1^2*t2^12 - 9*t2^14 + 10*t1^12*t2*t3 - 92*t1^10*t2^3*t3 + 310*t1^8*t2^5*t3 - 520*t1^6*t2^7*t3 + 470*t1^4*t2^9*t3 - 220*t1^2*t2^11*t3 + 42*t2^13*t3 - 15*t1^12*t3^2 + 206*t1^10*t2^2*t3^2 - 645*t1^8*t2^4*t3^2 + 820*t1^6*t2^6*t3^2 - 425*t1^4*t2^8*t3^2 + 30*t1^2*t2^10*t3^2 + 29*t2^12*t3^2 - 92*t1^10*t2*t3^3 + 340*t1^8*t2^3*t3^3 - 184*t1^6*t2^5*t3^3 - 568*t1^4*t2^7*t3^3 + 788*t1^2*t2^9*t3^3 - 284*t2^11*t3^3 + 69*t1^10*t3^4 - 645*t1^8*t2^2*t3^4 + 658*t1^6*t2^4*t3^4 + 310*t1^4*t2^6*t3^4 - 359*t1^2*t2^8*t3^4 - 33*t2^10*t3^4 + 310*t1^8*t2*t3^5 - 184*t1^6*t2^3*t3^5 - 316*t1^4*t2^5*t3^5 - 568*t1^2*t2^7*t3^5 + 758*t2^9*t3^5 - 155*t1^8*t3^6 + 820*t1^6*t2^2*t3^6 + 310*t1^4*t2^4*t3^6 + 548*t1^2*t2^6*t3^6 + 13*t2^8*t3^6 - 520*t1^6*t2*t3^7 - 568*t1^4*t2^3*t3^7 - 568*t1^2*t2^5*t3^7 - 1032*t2^7*t3^7 + 195*t1^6*t3^8 - 425*t1^4*t2^2*t3^8 - 359*t1^2*t2^4*t3^8 + 13*t2^6*t3^8 + 470*t1^4*t2*t3^9 + 788*t1^2*t2^3*t3^9 + 758*t2^5*t3^9 - 141*t1^4*t3^10 + 30*t1^2*t2^2*t3^10 - 33*t2^4*t3^10 - 220*t1^2*t2*t3^11 - 284*t2^3*t3^11 + 55*t1^2*t3^12 + 29*t2^2*t3^12 + 42*t2*t3^13 - 9*t3^14 + 10*t1^12*t2*t4 - 92*t1^10*t2^3*t4 + 310*t1^8*t2^5*t4 - 520*t1^6*t2^7*t4 + 470*t1^4*t2^9*t4 - 220*t1^2*t2^11*t4 + 42*t2^13*t4 + 10*t1^12*t3*t4 - 68*t1^10*t2^2*t3*t4 - 130*t1^8*t2^4*t3*t4 + 1000*t1^6*t2^6*t3*t4 - 1690*t1^4*t2^8*t3*t4 + 1180*t1^2*t2^10*t3*t4 - 302*t2^12*t3*t4 - 68*t1^10*t2*t3^2*t4 + 780*t1^8*t2^3*t3^2*t4 - 2184*t1^6*t2^5*t3^2*t4 + 2552*t1^4*t2^7*t3^2*t4 - 1332*t1^2*t2^9*t3^2*t4 + 252*t2^11*t3^2*t4 - 92*t1^10*t3^3*t4 + 780*t1^8*t2^2*t3^3*t4 - 856*t1^6*t2^4*t3^3*t4 + 1368*t1^4*t2^6*t3^3*t4 - 2828*t1^2*t2^8*t3^3*t4 + 1628*t2^10*t3^3*t4 - 130*t1^8*t2*t3^4*t4 - 856*t1^6*t2^3*t3^4*t4 - 140*t1^4*t2^5*t3^4*t4 + 2856*t1^2*t2^7*t3^4*t4 - 1730*t2^9*t3^4*t4 + 310*t1^8*t3^5*t4 - 2184*t1^6*t2^2*t3^5*t4 - 140*t1^4*t2^4*t3^5*t4 + 344*t1^2*t2^6*t3^5*t4 - 3450*t2^8*t3^5*t4 + 1000*t1^6*t2*t3^6*t4 + 1368*t1^4*t2^3*t3^6*t4 + 344*t1^2*t2^5*t3^6*t4 + 3560*t2^7*t3^6*t4 - 520*t1^6*t3^7*t4 + 2552*t1^4*t2^2*t3^7*t4 + 2856*t1^2*t2^4*t3^7*t4 + 3560*t2^6*t3^7*t4 - 1690*t1^4*t2*t3^8*t4 - 2828*t1^2*t2^3*t3^8*t4 - 3450*t2^5*t3^8*t4 + 470*t1^4*t3^9*t4 - 1332*t1^2*t2^2*t3^9*t4 - 1730*t2^4*t3^9*t4 + 1180*t1^2*t2*t3^10*t4 + 1628*t2^3*t3^10*t4 - 220*t1^2*t3^11*t4 + 252*t2^2*t3^11*t4 - 302*t2*t3^12*t4 + 42*t3^13*t4 - 15*t1^12*t4^2 + 206*t1^10*t2^2*t4^2 - 645*t1^8*t2^4*t4^2 + 820*t1^6*t2^6*t4^2 - 425*t1^4*t2^8*t4^2 + 30*t1^2*t2^10*t4^2 + 29*t2^12*t4^2 - 68*t1^10*t2*t3*t4^2 + 780*t1^8*t2^3*t3*t4^2 - 2184*t1^6*t2^5*t3*t4^2 + 2552*t1^4*t2^7*t3*t4^2 - 1332*t1^2*t2^9*t3*t4^2 + 252*t2^11*t3*t4^2 + 206*t1^10*t3^2*t4^2 - 2990*t1^8*t2^2*t3^2*t4^2 + 7884*t1^6*t2^4*t3^2*t4^2 - 9052*t1^4*t2^6*t3^2*t4^2 + 5382*t1^2*t2^8*t3^2*t4^2 - 1430*t2^10*t3^2*t4^2 + 780*t1^8*t2*t3^3*t4^2 - 3824*t1^6*t2^3*t3^3*t4^2 + 5704*t1^4*t2^5*t3^3*t4^2 - 2032*t1^2*t2^7*t3^3*t4^2 - 628*t2^9*t3^3*t4^2 - 645*t1^8*t3^4*t4^2 + 7884*t1^6*t2^2*t3^4*t4^2 - 9590*t1^4*t2^4*t3^4*t4^2 - 4900*t1^2*t2^6*t3^4*t4^2 + 5715*t2^8*t3^4*t4^2 - 2184*t1^6*t2*t3^5*t4^2 + 5704*t1^4*t2^3*t3^5*t4^2 + 7752*t1^2*t2^5*t3^5*t4^2 + 376*t2^7*t3^5*t4^2 + 820*t1^6*t3^6*t4^2 - 9052*t1^4*t2^2*t3^6*t4^2 - 4900*t1^2*t2^4*t3^6*t4^2 - 8628*t2^6*t3^6*t4^2 + 2552*t1^4*t2*t3^7*t4^2 - 2032*t1^2*t2^3*t3^7*t4^2 + 376*t2^5*t3^7*t4^2 - 425*t1^4*t3^8*t4^2 + 5382*t1^2*t2^2*t3^8*t4^2 + 5715*t2^4*t3^8*t4^2 - 1332*t1^2*t2*t3^9*t4^2 - 628*t2^3*t3^9*t4^2 + 30*t1^2*t3^10*t4^2 - 1430*t2^2*t3^10*t4^2 + 252*t2*t3^11*t4^2 + 29*t3^12*t4^2 - 92*t1^10*t2*t4^3 + 340*t1^8*t2^3*t4^3 - 184*t1^6*t2^5*t4^3 - 568*t1^4*t2^7*t4^3 + 788*t1^2*t2^9*t4^3 - 284*t2^11*t4^3 - 92*t1^10*t3*t4^3 + 780*t1^8*t2^2*t3*t4^3 - 856*t1^6*t2^4*t3*t4^3 + 1368*t1^4*t2^6*t3*t4^3 - 2828*t1^2*t2^8*t3*t4^3 + 1628*t2^10*t3*t4^3 + 780*t1^8*t2*t3^2*t4^3 - 3824*t1^6*t2^3*t3^2*t4^3 + 5704*t1^4*t2^5*t3^2*t4^3 - 2032*t1^2*t2^7*t3^2*t4^3 - 628*t2^9*t3^2*t4^3 + 340*t1^8*t3^3*t4^3 - 3824*t1^6*t2^2*t3^3*t4^3 + 2200*t1^4*t2^4*t3^3*t4^3 + 16208*t1^2*t2^6*t3

In [42]:
str(sqrtTvir(fixed_locus(1)[0],w=(x1,x2,x3,x4)))

'x1*x2*x3^-1*x4 + x1*x2^-1*x3*x4 + x1^-1*x2*x3*x4 - x1*x2^-1*x3^-1*x4 - x1^-1*x2*x3^-1*x4 - x1^-1*x2^-1*x3*x4 + x1^-1*x2^-1*x3^-1*x4'

In [19]:
list(x1+x2)

[(1, x1), (1, x2)]

In [21]:
list(character(fixed_locus(4)[3],v[2]))

[(1, x2^2*x3^-2), (1, 1), (1, x3^-2), (1, x3^-4)]

In [89]:
pretty(cut(Nek(1,v[0]))(x4=(x1*x2*x3)^-1))

(x1*x2 + 1)*(x1*x2 - 1)*(x1*x3 + 1)*(x1*x3 - 1)*(x2*x3 + 1)*(x2*x3 - 1)*w*(y - 1)/((x1*x2*x3 + 1)*(x1*x2*x3 - 1)*(x1 + 1)*(x1 - 1)*(x2 + 1)*(x2 - 1)*(x3 + 1)*(x3 - 1)*y)

In [121]:
pretty(denominator(cut(Nek_list(1,v,total[0]))))

-(x1*x2^3 - x3*x4)*(x1*x3^3 - x2*x4)*(x1*x2*x3*x4 - 1)*(x1*x4^3 - x2*x3)*(x2^3 - x1*x3*x4)*(x1*x2*x3 - x4^3)*(x1*x2*x3 - x4)*(x3^3 - x1*x2*x4)*(x1*x2*x4 - x3)*(x1*x3*x4 - x2)*(x2*x3*x4 - x1)*(x1*x2 - x3*x4)*(x1*x3 - x2*x4)*(x2*x3 - x1*x4)*u1*u2*u3*u4*x2*x3*x4*y

In [122]:
pretty(Nek(1,v[1]))

(x2^3 - x1*x3*x4)*(x1*x2*x3 - x4)*(x1*x2*x4 - x3)*(x1*x2 - x3*x4)*u2^3*w*x1*x3*x4*(y - 1)/((x1*x2^3 - x3*x4)*(x1*x3*x4 - x2)*(x1*x3 - x2*x4)*(x2*x3 - x1*x4)*u1*u3*u4*x2^3*y)

In [42]:
pretty(Nek(1,v[0]))

-(x1*x2 - x3*x4)*(x1*x3 - x2*x4)*(x2*x3 - x1*x4)*w*x1*x2*x3*x4*(y - 1)/((x1*x2*x3 - x4)*(x1*x2*x4 - x3)*(x1*x3*x4 - x2)*(x2*x3*x4 - x1)*u1*u2*u3*u4*y)

In [91]:
str(pretty(denominator(Nek(1))))

'-(x1*x2*x3 - x4)*(x1*x2*x4 - x3)*(x1*x3*x4 - x2)*(x2*x3*x4 - x1)*u1*u2*u3*u4*y'

In [3]:
pretty(Nek_list(1,v,total[0]))

8*w*(y - 1)/y

In [4]:
pretty(Nek_list(2,v,total[0]))

1/3*w^2*(y - 1)^2/y^2
-w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
1/3*w^2*(y - 1)^2/y^2
-w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
3/5*w^2*(y - 1)^2/y^2
-3/5*w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
3/5*w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
-1/3*w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
1/3*w^2*(y - 1)^2/y^2
-w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
1/3*w^2*(y - 1)^2/y^2
-w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
3/5*w^2*(y - 1)^2/y^2
-3/5*w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
3/5*w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
-1/3*w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2
w^2*(y - 1)^2/y^2


208/15*w^2*(y - 1)^2/y^2

In [2]:
pretty(Nek_list(1,v,total[0])(x1=1,x2=1,x3=1,u1=1,u2=1,u3=1,u4=1)(x4=1))

8*w*(y - 1)/y

In [74]:
for i in range(len(total)):
    print(pretty(cut(Nek_list(1,v,total[i]))(x2=1,x3=1,x4=1,u1=1,u2=1,u3=1,u4=1)))

0
-2*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-6*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-6*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-6*w*x1*(y - 1)/((x1 - 1)*y)
-6*w*x1*(y - 1)/((x1 - 1)*y)
-8*w*x1*(y - 1)/((x1 - 1)*y)
2*w*x1*(y - 1)/((x1 - 1)*y)
0
0
-2*w*x1*(y - 1)/((x1 - 1)*y)
0
-2*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
0
-2*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
-6*w*x1*(y - 1)/((x1 - 1)*y)
2*w*x1*(y - 1)/((x1 - 1)*y)
0
0
-2*w*x1*(y - 1)/((x1 - 1)*y)
0
-2*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y)
-4*w*x1*(y - 1)/((x1 - 1)*y)
0
-2*w*x1*(y - 1)/((x1 - 1)*y)
-2*w*x1*(y - 1)/((x1 - 1)*y

In [71]:
pretty(cut(Nek_list(1,v,total[15]))(x2=1,x3=1,x4=1,u1=1,u2=1,u3=1,u4=1))

-8*w*x1*(y - 1)/((x1 - 1)*y)

In [69]:
pretty(Nek(1,v[0]))

-(x1*x2 - x3*x4)*(x1*x3 - x2*x4)*(x2*x3 - x1*x4)*w*x1*x2*x3*x4*(y - 1)/((x1*x2*x3 - x4)*(x1*x2*x4 - x3)*(x1*x3*x4 - x2)*(x2*x3*x4 - x1)*u1*u2*u3*u4*y)

In [70]:
NekBracket(x1)

(x1*u1 - u1)/x1

In [163]:
v=w[4]
sqrtTvir(fixed_locus(2)[1],v)+dual(sqrtTvir(fixed_locus(2)[1],v))-Tvir(fixed_locus(2)[1],v)(x4=x4)

0

In [156]:
Tvir(fixed_locus(2)[0],w[1])(x4=x4)

x1^2*x2^10*x3^-6*x4^-2 + x1^2*x2^6*x3^-2*x4^-2 - x1^2*x2^2*x3^2*x4^-2 - x1^2*x2^-2*x3^6*x4^-2 - x1^2*x2^6*x3^-6*x4^2 - x1^2*x2^2*x3^-2*x4^2 + x1^2*x2^-2*x3^2*x4^2 + x1^2*x2^-6*x3^6*x4^2 - x1^2*x2^6*x3^-6*x4^-2 - x1^2*x2^2*x3^-2*x4^-2 + x1^2*x2^-2*x3^2*x4^-2 + x1^-2*x2^2*x3^2*x4^-2 + x1^2*x2^-6*x3^6*x4^-2 + x1^-2*x2^-2*x3^6*x4^-2 + x1^2*x2^2*x3^-6*x4^2 + x1^-2*x2^6*x3^-6*x4^2 + x1^2*x2^-2*x3^-2*x4^2 + x1^-2*x2^2*x3^-2*x4^2 - x1^-2*x2^-2*x3^2*x4^2 - x1^-2*x2^-6*x3^6*x4^2 + x1^-2*x2^6*x3^-6*x4^-2 + x1^-2*x2^2*x3^-2*x4^-2 - x1^-2*x2^-2*x3^2*x4^-2 - x1^-2*x2^-6*x3^6*x4^-2 - x1^-2*x2^2*x3^-6*x4^2 - x1^-2*x2^-2*x3^-2*x4^2 + x1^-2*x2^-6*x3^2*x4^2 + x1^-2*x2^-10*x3^6*x4^2

In [58]:
dual(Tvir(fixed_locus(1)[0],w[0])(x4=x4))

x1^2*x2^2*x3^2*x4^-2 + x1^2*x2^2*x3^-2*x4^2 + x1^2*x2^-2*x3^2*x4^2 + x1^-2*x2^2*x3^2*x4^2 - x1^2*x2^2*x3^-2*x4^-2 - x1^2*x2^-2*x3^2*x4^-2 - x1^-2*x2^2*x3^2*x4^-2 - x1^2*x2^-2*x3^-2*x4^2 - x1^-2*x2^2*x3^-2*x4^2 - x1^-2*x2^-2*x3^2*x4^2 + x1^2*x2^-2*x3^-2*x4^-2 + x1^-2*x2^2*x3^-2*x4^-2 + x1^-2*x2^-2*x3^2*x4^-2 + x1^-2*x2^-2*x3^-2*x4^2

In [51]:
pretty(NekBracket(Tvir(fixed_locus(1)[0],w[1])(x4=x4)))

-(x1*x2^3 + x3*x4)^2*(x1*x2^3 - x3*x4)^2*(x1*x3*x4 + x2)^2*(x1*x3*x4 - x2)^2*(x1*x3 + x2*x4)^2*(x1*x3 - x2*x4)^2*(x2*x3 + x1*x4)^2*(x2*x3 - x1*x4)^2/((x1*x2*x3 + x4)^2*(x1*x2*x3 - x4)^2*(x1*x2*x4 + x3)^2*(x1*x2*x4 - x3)^2*(x1*x2 + x3*x4)^2*(x1*x2 - x3*x4)^2*x1^2*x2^6*x3^2*x4^2)

In [38]:
pretty(NekBracket(x1+x1^-1))

-(x1 - 1)^2/x1

In [23]:
Tvir(fixed_locus(1)[0],w[4])

-x1*x2*x3 + x1*x2 + x1*x3 + x2*x3 - x2 - x3 + 2 - x3^-1 - x2^-1 + x2^-1*x3^-1 + x1^-1*x3^-1 + x1^-1*x2^-1 - x1^-1*x2^-1*x3^-1

In [367]:
dual(Tvir(fixed_locus(1)[0],w[0]))

x1*x2*x3 - x1*x2 - x1*x3 - x2*x3 + x1 + x2 + x3 + x3^-1 + x2^-1 + x1^-1 - x2^-1*x3^-1 - x1^-1*x3^-1 - x1^-1*x2^-1 + x1^-1*x2^-1*x3^-1

In [362]:
Tvir(fixed_locus(1)[0])

x1*x2*x3 - x1*x2 - x1*x3 - x2*x3 + x1 + x2 + x3 + x3^-1 + x2^-1 + x1^-1 - x2^-1*x3^-1 - x1^-1*x3^-1 - x1^-1*x2^-1 + x1^-1*x2^-1*x3^-1

In [134]:

cut(Nek(1,w))

(x1^2*x2^2*x3^2*y^8 - 8*x1^2*x2^2*x3^2*y^7 + 28*x1^2*x2^2*x3^2*y^6 - x1^2*x2*x3*y^8 - x1*x2^2*x3*y^8 - x1*x2*x3^2*y^8 - 56*x1^2*x2^2*x3^2*y^5 + 8*x1^2*x2*x3*y^7 + 8*x1*x2^2*x3*y^7 + 8*x1*x2*x3^2*y^7 + 70*x1^2*x2^2*x3^2*y^4 - 28*x1^2*x2*x3*y^6 - 28*x1*x2^2*x3*y^6 - 28*x1*x2*x3^2*y^6 + x1*x2*y^8 + x1*x3*y^8 + x2*x3*y^8 - 56*x1^2*x2^2*x3^2*y^3 + 56*x1^2*x2*x3*y^5 + 56*x1*x2^2*x3*y^5 + 56*x1*x2*x3^2*y^5 - 8*x1*x2*y^7 - 8*x1*x3*y^7 - 8*x2*x3*y^7 + 28*x1^2*x2^2*x3^2*y^2 - 70*x1^2*x2*x3*y^4 - 70*x1*x2^2*x3*y^4 - 70*x1*x2*x3^2*y^4 + 28*x1*x2*y^6 + 28*x1*x3*y^6 + 28*x2*x3*y^6 - y^8 - 8*x1^2*x2^2*x3^2*y + 56*x1^2*x2*x3*y^3 + 56*x1*x2^2*x3*y^3 + 56*x1*x2*x3^2*y^3 - 56*x1*x2*y^5 - 56*x1*x3*y^5 - 56*x2*x3*y^5 + 8*y^7 + x1^2*x2^2*x3^2 - 28*x1^2*x2*x3*y^2 - 28*x1*x2^2*x3*y^2 - 28*x1*x2*x3^2*y^2 + 70*x1*x2*y^4 + 70*x1*x3*y^4 + 70*x2*x3*y^4 - 28*y^6 + 8*x1^2*x2*x3*y + 8*x1*x2^2*x3*y + 8*x1*x2*x3^2*y - 56*x1*x2*y^3 - 56*x1*x3*y^3 - 56*x2*x3*y^3 + 56*y^5 - x1^2*x2*x3 - x1*x2^2*x3 - x1*x2*x3^2 + 28*x1*x2*y^2 + 28*x1*x3*y^2 + 28*x2*x3*y^2 - 70*y^4 - 8*x1*x2*y - 8*x1*x3*y - 8*x2*x3*y + 56*y^3 + x1*x2 + x1*x3 + x2*x3 - 28*y^2 + 8*y - 1)/(x1^2*x2^2*x3^2*y^4 - x1^2*x2^2*x3*y^4 - x1^2*x2*x3^2*y^4 - x1*x2^2*x3^2*y^4 + x1^2*x2*x3*y^4 + x1*x2^2*x3*y^4 + x1*x2*x3^2*y^4 - 2*x1*x2*x3*y^4 + x1*x2*y^4 + x1*x3*y^4 + x2*x3*y^4 - x1*y^4 - x2*y^4 - x3*y^4 + y^4)

In [104]:
pretty(sqrt_to_original(Nek(1).numerator()))/pretty(sqrt_to_original(Nek(1).denominator()))

(x1*x2 - 1)*(x1*x3 - 1)*(x2*x3 - 1)*w*(y - 1)/((x1*x2*x3 - 1)*(x1 - 1)*(x2 - 1)*(x3 - 1)*y)

In [103]:
pretty(sqrt_to_original(Nek(2).numerator()))/pretty(sqrt_to_original(Nek(2).denominator()))

(2*x1^4*x2^4*x3^4*y + x1^4*x2^4*x3^4 - x1^4*x2^4*x3^3 - x1^4*x2^3*x3^4 - x1^3*x2^4*x3^4 - x1^4*x2^4*x3^2*y + x1^4*x2^3*x3^3*y + x1^3*x2^4*x3^3*y - x1^4*x2^2*x3^4*y + x1^3*x2^3*x3^4*y - x1^2*x2^4*x3^4*y - x1^4*x2^4*x3^2 + x1^4*x2^3*x3^3 + x1^3*x2^4*x3^3 - x1^4*x2^2*x3^4 + x1^3*x2^3*x3^4 - x1^2*x2^4*x3^4 - x1^4*x2^3*x3^2*y - x1^3*x2^4*x3^2*y - x1^4*x2^2*x3^3*y - 3*x1^3*x2^3*x3^3*y - x1^2*x2^4*x3^3*y - x1^3*x2^2*x3^4*y - x1^2*x2^3*x3^4*y - 2*x1^3*x2^3*x3^3 + x1^4*x2^2*x3^2*y + 2*x1^3*x2^3*x3^2*y + x1^2*x2^4*x3^2*y + 2*x1^3*x2^2*x3^3*y + 2*x1^2*x2^3*x3^3*y + x1^2*x2^2*x3^4*y + 2*x1^4*x2^2*x3^2 + 2*x1^3*x2^3*x3^2 + 2*x1^2*x2^4*x3^2 + 2*x1^3*x2^2*x3^3 + 2*x1^2*x2^3*x3^3 + 2*x1^2*x2^2*x3^4 - 2*x1^3*x2^2*x3^2*y - 2*x1^2*x2^3*x3^2*y - 2*x1^2*x2^2*x3^3*y - x1^3*x2^3*x3 - 3*x1^3*x2^2*x3^2 - 3*x1^2*x2^3*x3^2 - x1^3*x2*x3^3 - 3*x1^2*x2^2*x3^3 - x1*x2^3*x3^3 + x1^3*x2^2*x3*y + x1^2*x2^3*x3*y + x1^3*x2*x3^2*y + 2*x1^2*x2^2*x3^2*y + x1*x2^3*x3^2*y + x1^2*x2*x3^3*y + x1*x2^2*x3^3*y + x1^3*x2^2*x3 + x1^2*x2^3*x3 + x1^3*x2*x3^2 + 2*x1^2*x2^2*x3^2 + x1*x2^3*x3^2 + x1^2*x2*x3^3 + x1*x2^2*x3^3 - x1^3*x2*x3*y - 3*x1^2*x2^2*x3*y - x1*x2^3*x3*y - 3*x1^2*x2*x3^2*y - 3*x1*x2^2*x3^2*y - x1*x2*x3^3*y - 2*x1^2*x2^2*x3 - 2*x1^2*x2*x3^2 - 2*x1*x2^2*x3^2 + 2*x1^2*x2^2*y + 2*x1^2*x2*x3*y + 2*x1*x2^2*x3*y + 2*x1^2*x3^2*y + 2*x1*x2*x3^2*y + 2*x2^2*x3^2*y + x1^2*x2^2 + 2*x1^2*x2*x3 + 2*x1*x2^2*x3 + x1^2*x3^2 + 2*x1*x2*x3^2 + x2^2*x3^2 - 2*x1*x2*x3*y - x1^2*x2 - x1*x2^2 - x1^2*x3 - 3*x1*x2*x3 - x2^2*x3 - x1*x3^2 - x2*x3^2 - x1^2*y + x1*x2*y - x2^2*y + x1*x3*y + x2*x3*y - x3^2*y - x1^2 + x1*x2 - x2^2 + x1*x3 + x2*x3 - x3^2 - x1*y - x2*y - x3*y + y + 2)*(x1*x2 - 1)*(x1*x3 - 1)*(x2*x3 - 1)*(y - 1)/((x1*x2*x3 + 1)*(x1*x2*x3 - 1)^2*(x1 + 1)*(x1 - 1)^2*(x2 + 1)*(x2 - 1)^2*(x3 + 1)*(x3 - 1)^2*y)

In [105]:
str(pretty(sqrt_to_original(Nek(3).numerator()))/pretty(sqrt_to_original(Nek(3).denominator())))

KeyboardInterrupt: 

In [67]:
pretty(Nek(1))

(x1*x2 - 1)*(x1*x3 - 1)*(x2*x3 - 1)*w*(y - 1)/((x1*x2*x3 - 1)*(x1 - 1)*(x2 - 1)*(x3 - 1)*y)

In [19]:
n,N,r=1,1,1
S=Segre_Num_quot_fast(n,N,r)
V=Verlinde_Num_quot_fast(n,N,r)
diff= S-(-1)^(N*n)*V
pretty(diff)

0

In [20]:
n,N,r=2,1,1
S=Segre_Num_quot_fast(n,N,r)
V=Verlinde_Num_quot_fast(n,N,r)
diff= S-(-1)^(N*n)*V
pretty(diff)

1/2*(m1 + w1)^2*(t1 + t2)^2*(t1 + t3)^2*(t2 + t3)^2/((t1 + t2 + t3)^2*t1^2*t2^2*t3^2)

In [21]:
n,N,r=3,1,1
S=Segre_Num_quot_fast(n,N,r)
V=Verlinde_Num_quot_fast(n,N,r)
diff= S-(-1)^(N*n)*V
pretty(diff)

-1/24*(22*m1^3*t1^2*t2 + 22*m1^3*t1*t2^2 + 22*m1^3*t1^2*t3 + 44*m1^3*t1*t2*t3 - 195*m1^2*t1^2*t2*t3 - 24*t1^4*t2*t3 + 22*m1^3*t2^2*t3 - 195*m1^2*t1*t2^2*t3 - 48*t1^3*t2^2*t3 - 48*t1^2*t2^3*t3 - 24*t1*t2^4*t3 + 22*m1^3*t1*t3^2 + 22*m1^3*t2*t3^2 - 195*m1^2*t1*t2*t3^2 - 48*t1^3*t2*t3^2 - 72*t1^2*t2^2*t3^2 - 48*t1*t2^3*t3^2 - 48*t1^2*t2*t3^3 - 48*t1*t2^2*t3^3 - 24*t1*t2*t3^4 + 66*m1^2*t1^2*t2*w1 + 66*m1^2*t1*t2^2*w1 + 66*m1^2*t1^2*t3*w1 + 132*m1^2*t1*t2*t3*w1 - 390*m1*t1^2*t2*t3*w1 + 66*m1^2*t2^2*t3*w1 - 390*m1*t1*t2^2*t3*w1 + 66*m1^2*t1*t3^2*w1 + 66*m1^2*t2*t3^2*w1 - 390*m1*t1*t2*t3^2*w1 + 66*m1*t1^2*t2*w1^2 + 66*m1*t1*t2^2*w1^2 + 66*m1*t1^2*t3*w1^2 + 132*m1*t1*t2*t3*w1^2 - 195*t1^2*t2*t3*w1^2 + 66*m1*t2^2*t3*w1^2 - 195*t1*t2^2*t3*w1^2 + 66*m1*t1*t3^2*w1^2 + 66*m1*t2*t3^2*w1^2 - 195*t1*t2*t3^2*w1^2 + 22*t1^2*t2*w1^3 + 22*t1*t2^2*w1^3 + 22*t1^2*t3*w1^3 + 44*t1*t2*t3*w1^3 + 22*t2^2*t3*w1^3 + 22*t1*t3^2*w1^3 + 22*t2*t3^2*w1^3)*(t1 + t2)^2*(t1 + t3)^2*(t2 + t3)^2/((t1 + t2 + t3)^3*t1^3*t2^3*t3^3)

In [23]:
n,N,r=2,2,1
S=Segre_Num_quot_fast(n,N,r)
V=Verlinde_Num_quot_fast(n,N,r)
diff= S-(-1)^(N*n)*V
pretty(diff)

(m1 + w1)*(m2 + w1)*(t1 + t2)^2*(t1 + t3)^2*(t2 + t3)^2/((t1 + t2 + t3)^2*t1^2*t2^2*t3^2)

In [24]:
n,N,r=2,2,2
S=Segre_Num_quot_fast(n,N,r)
V=Verlinde_Num_quot_fast(n,N,r)
diff= S-(-1)^(N*n)*V
pretty(diff)

1/2*(m1^2 + 10*m1*m2 + m2^2 + 6*m1*w1 + 6*m2*w1 + 7*w1^2 + 6*m1*w2 + 6*m2*w2 - 2*w1*w2 + 7*w2^2)*(t1 + t2)^2*(t1 + t3)^2*(t2 + t3)^2/((t1 + t2 + t3)^2*t1^2*t2^2*t3^2)